In [1]:
from pybaseball import statcast
import pandas as pd
import numpy as np

In [ ]:
raw_data = statcast(start_dt="2021-04-01", end_dt="2021-11-02")
raw_data
raw_data.to_csv('C:/Users/Brandon Loesch/Desktop/Machine Learning/Project/data/raw_pitch_data_2021.csv')

This is a large query, it may take a moment to complete


 27%|██▋       | 59/216 [00:52<02:22,  1.10it/s]

In [14]:
batted_balls = (raw_data.loc[(raw_data['description'] =='hit_into_play') &
                          (raw_data['game_type'] =='R')])
batted_balls

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2880,FF,2021-10-03,92.3,1.4,6.8,"Smith, Will",596019,519293,field_out,hit_into_play,...,5,0,5,0,5,Infield shift,Strategic,148,0.0,-0.073
3586,FF,2021-10-03,91.2,1.49,6.66,"Smith, Will",607043,519293,field_out,hit_into_play,...,5,0,5,0,5,Standard,Standard,143,0.002,-0.189
2326,SI,2021-10-03,94.3,-1.2,6.05,"Gsellman, Robert",594807,607229,field_out,hit_into_play,...,0,0,5,5,0,Infield shift,Standard,221,0.0,-0.061
3657,SL,2021-10-03,90.5,-1.51,5.93,"Gsellman, Robert",645277,607229,field_out,hit_into_play,...,0,0,5,5,0,Infield shift,Standard,216,-0.001,-0.174
2551,FF,2021-10-03,94.9,-2.71,6.38,"Martin, Chris",606299,455119,field_out,hit_into_play,...,5,0,5,0,5,Standard,Standard,216,0.003,-0.163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,FF,2021-04-01,91.2,1.47,6.24,"Kershaw, Clayton",606132,477132,field_out,hit_into_play,...,0,0,0,0,0,Standard,Strategic,178,-0.021,-0.35
1742,FF,2021-04-01,97.1,-1.83,6.26,"Márquez, Germán",571970,608566,field_out,hit_into_play,...,0,0,0,0,0,Standard,Standard,201,0.042,-0.39
2765,SI,2021-04-01,95.8,-1.7,6.25,"Márquez, Germán",457759,608566,grounded_into_double_play,hit_into_play,...,0,0,0,0,0,Standard,Standard,209,0.103,-0.566
3198,FF,2021-04-01,96.2,-2.14,6.08,"Márquez, Germán",608369,608566,single,hit_into_play,...,0,0,0,0,0,Infield shift,Standard,216,-0.052,0.701


In [8]:
called_strike = (raw_data.loc[(raw_data['description'] =='called_strike') &
                          (raw_data['game_type'] =='R')])
called_strike['sz_vert'] = called_strike['sz_top'] - called_strike['sz_bot']
called_strike['plate_z_norm'] = called_strike['plate_z'] - called_strike['sz_vert']


C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_11368\2307598387.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  called_strike['sz_vert'] = called_strike['sz_top'] - called_strike['sz_bot']
C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_11368\2307598387.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  called_strike['plate_z_norm'] = called_strike['plate_z'] - called_strike['sz_vert']


2.4099999999999997

In [31]:
print(called_strike['plate_z_norm'].max(),
called_strike['plate_z_norm'].min(),
called_strike['plate_z_norm'].mean(),
called_strike['plate_z'].mean())

2.4099999999999997 -1.92 0.5768207398256813 2.3654704550133827


Visualization of pitch zones can be seen here: https://baseballsavant.mlb.com/site-core/images/attack-zone.png

In [15]:
data = batted_balls[['pitch_type', 'release_speed', 'stand', 'p_throws', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'sz_top', 'sz_bot', 'effective_speed', 'release_spin_rate', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle']].copy()
data

,pitch_type,release_speed,stand,p_throws,pfx_x,pfx_z,plate_x,plate_z,sz_top,sz_bot,effective_speed,release_spin_rate,estimated_ba_using_speedangle,estimated_woba_using_speedangle
2880,FF,92.3,R,L,0.69,1.38,-0.69,2.83,3.32,1.51,92.8,2330,0.174,0.158
3586,FF,91.2,L,L,0.63,1.28,-0.31,2.8,3.49,1.6,90.9,2281,0.1,0.09
2326,SI,94.3,R,R,-1.35,0.88,-0.26,1.78,3.41,1.56,94.4,1982,0.0,0.0
3657,SL,90.5,L,R,0.01,0.62,0.01,2.23,3.26,1.48,91.1,2133,0.409,0.457
2551,FF,94.9,R,R,-0.75,1.18,-0.1,1.94,3.37,1.53,95.5,2224,0.109,0.156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,FF,91.2,L,L,-0.03,1.38,-0.74,3.0,3.49,1.6,92.1,2535,0.094,0.091
1742,FF,97.1,L,R,-0.26,0.72,-0.05,2.33,3.37,1.53,96.8,2233,0.071,0.064
2765,SI,95.8,R,R,-1.1,0.47,0.09,0.88,3.32,1.51,94.5,2207,0.229,0.211
3198,FF,96.2,L,R,-0.24,0.79,-0.15,2.49,3.55,1.63,95.4,2165,0.577,0.519


Here pfx_x and pfx_z columns are both in feet, these should be converted to inches to get a better idea of what's going on.

Also, pfx_x is from the catcher's point of view so this will be changed to be from the pitcher's point of view.

X is the horizontal movement. A positive value indicates a pitch that runs to the right side of the box and a negative value indicates a pitch that runs to the left side of the box. 

And Z is the verical movement. A positive value indicates a pitch that rises/counteracts gravity (fastball) and a negative value indicates a pitch that falls (sinker).

sz_top and sz_bot represent the height of the strikezone depending on the hitter, generally between 3.5 to 1.5.

The width of the strikezone is generally between -0.71 and 0.71 feet, but will depend on who is the umpire.

An overall movement will be calculated for each pitch as well.

In [16]:
data['pfx_x'] = data['pfx_x'] * -12 #convert to inches
data['pfx_z'] = data['pfx_z'] * 12 #convert to inches
data['tot_movement'] = np.sqrt((data['pfx_x']**2) + (data['pfx_z']**2))

data.rename(columns={'pfx_x' : 'hor_movement', 'pfx_z' : 'vert_movement'})

,pitch_type,release_speed,stand,p_throws,hor_movement,vert_movement,plate_x,plate_z,sz_top,sz_bot,effective_speed,release_spin_rate,estimated_ba_using_speedangle,estimated_woba_using_speedangle,tot_movement
2880,FF,92.3,R,L,-8.28,16.56,-0.69,2.83,3.32,1.51,92.8,2330,0.174,0.158,18.514643
3586,FF,91.2,L,L,-7.56,15.36,-0.31,2.8,3.49,1.6,90.9,2281,0.1,0.09,17.119673
2326,SI,94.3,R,R,16.2,10.56,-0.26,1.78,3.41,1.56,94.4,1982,0.0,0.0,19.33788
3657,SL,90.5,L,R,-0.12,7.44,0.01,2.23,3.26,1.48,91.1,2133,0.409,0.457,7.440968
2551,FF,94.9,R,R,9.0,14.16,-0.1,1.94,3.37,1.53,95.5,2224,0.109,0.156,16.778129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,FF,91.2,L,L,0.36,16.56,-0.74,3.0,3.49,1.6,92.1,2535,0.094,0.091,16.563913
1742,FF,97.1,L,R,3.12,8.64,-0.05,2.33,3.37,1.53,96.8,2233,0.071,0.064,9.186076
2765,SI,95.8,R,R,13.2,5.64,0.09,0.88,3.32,1.51,94.5,2207,0.229,0.211,14.354428
3198,FF,96.2,L,R,2.88,9.48,-0.15,2.49,3.55,1.63,95.4,2165,0.577,0.519,9.907815


Plate_x is represented as horizontal distance away from the center of the strikezone, but plate_z is not. To change this, the height of the strikezone will be calculated then the vertical distance away from the center will be calculated. This represents plate_x and plate_z in a similar way.

In [28]:
data['sz_vert'] = data['sz_top'] - data['sz_bot']
data['sz_mid'] = data['sz_vert'] / 2
data['plate_z_norm'] = (data['plate_z'] - data['sz_mid'])
data['plate_z_norm'].mean()

1.4267183569796307

In [189]:
cleandf = data[['pitch_type', 'release_speed', 'stand', 'p_throws', 'pfx_x', 'pfx_z', 'tot_movement', 'plate_x', 'plate_z_norm', 'effective_speed', 'release_spin_rate', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle']]
cleandf

,pitch_type,release_speed,stand,p_throws,pfx_x,pfx_z,tot_movement,plate_x,plate_z_norm,effective_speed,release_spin_rate,estimated_ba_using_speedangle,estimated_woba_using_speedangle
2933,FF,92.3,R,L,-8.28,16.56,18.514643,-0.69,1.02,92.8,2330,0.174,0.158
3485,FF,91.2,L,L,-7.56,15.36,17.119673,-0.31,0.91,90.9,2281,0.1,0.09
2319,SI,94.3,R,R,16.2,10.56,19.33788,-0.26,-0.07,94.4,1982,0.0,0.0
3756,SL,90.5,L,R,-0.12,7.44,7.440968,0.01,0.45,91.1,2133,0.409,0.457
2469,FF,94.9,R,R,9.0,14.16,16.778129,-0.1,0.1,95.5,2224,0.109,0.156
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3828,FF,91.2,L,L,0.36,16.56,16.563913,-0.74,1.11,92.1,2535,0.094,0.091
1754,FF,97.1,L,R,3.12,8.64,9.186076,-0.05,0.49,96.8,2233,0.071,0.064
2766,SI,95.8,R,R,13.2,5.64,14.354428,0.09,-0.93,94.5,2207,0.229,0.211
3173,FF,96.2,L,R,2.88,9.48,9.907815,-0.15,0.57,95.4,2165,0.577,0.519


We have 3 categories of data in this dataset that will be used for predictors: 1) categorical, 2) location and 3) attributes.

1) Pitcher/batter handedness, pitch type

2) Horizontal/vertical location

3) Speed, movement, spin rate

Lastly we will be predicting estimated_ba_using_speedangle and estimated_woba_using_speedangle. These are explained in the document.

In [191]:
cleandf.to_csv('C:/Users/Brandon Loesch/Desktop/Machine Learning/Project/data/pitch_data_2021.csv')

In [36]:
from pybaseball import cache

cache.enable()

In [37]:
new_data = statcast(start_dt="2022-06-01", end_dt="2022-8-31")
new_data

This is a large query, it may take a moment to complete


100%|██████████| 92/92 [00:53<00:00,  1.72it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2054,CH,2022-08-31,84.2,-2.34,4.92,"Herget, Jimmy",665828,623474,field_out,hit_into_play,...,3,2,3,2,3,Infield shift,Standard,267,0.14,-0.384
2120,SL,2022-08-31,82.9,-2.43,4.89,"Herget, Jimmy",665828,623474,NaN,foul,...,3,2,3,2,3,Infield shift,Standard,94,0.0,-0.061
2157,SL,2022-08-31,83.0,-2.35,4.81,"Herget, Jimmy",650402,623474,strikeout,swinging_strike,...,3,2,3,2,3,Standard,Standard,92,0.135,-0.326
2213,CU,2022-08-31,72.2,-2.64,4.03,"Herget, Jimmy",650402,623474,NaN,ball,...,3,2,3,2,3,Standard,Standard,98,0.0,0.03
2264,CU,2022-08-31,74.2,-2.61,4.65,"Herget, Jimmy",650402,623474,NaN,foul,...,3,2,3,2,3,Standard,Standard,78,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912,CU,2022-06-01,84.1,-2.02,6.54,"Bradish, Kyle",608385,680694,strikeout,called_strike,...,0,0,0,0,0,Infield shift,Standard,17,0.022,-0.195
3946,SL,2022-06-01,88.1,-2.22,6.42,"Bradish, Kyle",608385,680694,NaN,ball,...,0,0,0,0,0,Infield shift,Standard,47,0.0,0.037
4084,FF,2022-06-01,97.9,-2.26,6.41,"Bradish, Kyle",608385,680694,NaN,ball,...,0,0,0,0,0,Infield shift,Standard,186,0.0,0.017
4258,FF,2022-06-01,97.8,-2.22,6.44,"Bradish, Kyle",608385,680694,NaN,foul,...,0,0,0,0,0,Infield shift,Standard,185,0.0,-0.046


In [38]:
new_bb = (new_data.loc[(new_data['description'] =='hit_into_play') &
                          (new_data['game_type'] =='R') ])
new_bb

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2054,CH,2022-08-31,84.2,-2.34,4.92,"Herget, Jimmy",665828,623474,field_out,hit_into_play,...,3,2,3,2,3,Infield shift,Standard,267,0.14,-0.384
3633,SI,2022-08-31,96.1,-0.69,6.51,"Holmes, Clay",545361,605280,field_out,hit_into_play,...,2,2,3,3,2,Infield shift,Standard,230,-0.009,-0.178
2354,SL,2022-08-31,81.4,2.05,5.27,"Quijada, José",643217,650671,field_out,hit_into_play,...,3,2,3,2,3,Standard,Standard,295,0.058,-0.219
3310,FF,2022-08-31,96.2,1.92,5.42,"Quijada, José",643396,650671,field_out,hit_into_play,...,3,2,3,2,3,Standard,Standard,151,0.039,-0.134
8,FF,2022-08-31,97.5,-1.66,5.78,"Cole, Gerrit",545358,543037,field_out,hit_into_play,...,2,2,3,3,2,Standard,Standard,212,-0.009,-0.178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,FF,2022-06-01,96.7,-2.21,6.43,"Bradish, Kyle",666211,680694,field_out,hit_into_play,...,0,0,0,0,0,Infield shift,Standard,190,0.024,-0.295
2039,FF,2022-06-01,95.2,-2.22,6.5,"Bradish, Kyle",624428,680694,field_out,hit_into_play,...,0,0,0,0,0,Standard,Standard,189,0.017,-0.156
3493,FF,2022-06-01,94.7,2.5,6.28,"Ray, Robbie",641820,592662,field_out,hit_into_play,...,0,0,0,0,0,Standard,Standard,148,-0.016,-0.148
2601,SL,2022-06-01,86.4,-2.18,6.45,"Bradish, Kyle",641487,680694,field_out,hit_into_play,...,0,0,0,0,0,Standard,Standard,40,0.022,-0.209


In [39]:
new_df = new_bb[['pitcher', 'pitch_type', 'release_speed', 'stand', 'p_throws', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'sz_top', 'sz_bot', 'effective_speed', 'release_spin_rate', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle']].copy()
new_df

,pitcher,pitch_type,release_speed,stand,p_throws,pfx_x,pfx_z,plate_x,plate_z,sz_top,sz_bot,effective_speed,release_spin_rate,estimated_ba_using_speedangle,estimated_woba_using_speedangle
2054,623474,CH,84.2,L,R,-0.67,-0.46,-1.45,1.89,3.18,1.49,83.0,1533,0.049,0.044
3633,605280,SI,96.1,R,R,-1.43,0.67,-0.96,2.89,3.49,1.65,95.5,2234,0.663,0.597
2354,650671,SL,81.4,L,L,-0.94,-0.05,0.14,2.51,3.28,1.48,81.2,2538,0.006,0.005
3310,650671,FF,96.2,R,L,0.47,1.17,-0.86,2.68,3.1,1.4,97.1,2239,0.134,0.123
8,543037,FF,97.5,R,R,-0.86,1.53,0.11,2.66,3.5,1.67,97.3,2402,0.009,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,680694,FF,96.7,L,R,0.02,1.29,-0.38,2.36,3.54,1.65,97.7,2286,0.34,0.545
2039,680694,FF,95.2,L,R,0.01,1.24,-0.18,2.45,3.14,1.48,96.0,2259,0.223,0.201
3493,592662,FF,94.7,R,L,0.77,1.43,0.38,2.66,3.65,1.71,94.8,2395,0.071,0.064
2601,680694,SL,86.4,L,R,1.2,-0.17,0.36,1.88,3.36,1.56,86.7,2560,0.034,0.031


In [40]:
new_df['pfx_x'] = new_df['pfx_x'] * -12 #convert to inches
new_df['pfx_z'] = new_df['pfx_z'] * 12 #convert to inches
new_df['tot_movement'] = np.sqrt((new_df['pfx_x']**2) + (new_df['pfx_z']**2))

new_df.rename(columns={'pfx_x' : 'hor_movement', 'pfx_z' : 'vert_movement'})

new_df['sz_vert'] = new_df['sz_top'] - new_df['sz_bot']
new_df['sz_mid'] = new_df['sz_vert'] / 2
new_df['plate_z_norm'] = (new_df['plate_z'] - new_df['sz_mid'])

new_cleandf = new_df[['pitcher','pitch_type', 'release_speed', 'stand', 'p_throws', 'pfx_x', 'pfx_z', 'tot_movement', 'plate_x', 'plate_z_norm', 'effective_speed', 'release_spin_rate', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle']]
new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'ST'].index, inplace=True)
new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'SV'].index, inplace=True)
new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'CS'].index, inplace=True)
new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'EP'].index, inplace=True)
new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'KN'].index, inplace=True)
new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'SC'].index, inplace=True)

new_cleandf = new_cleandf.dropna()
new_cleandf

C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_21192\3762259000.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'ST'].index, inplace=True)
C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_21192\3762259000.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_cleandf.drop(new_cleandf[new_cleandf['pitch_type'] == 'SV'].index, inplace=True)
C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_21192\3762259000.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

,pitcher,pitch_type,release_speed,stand,p_throws,pfx_x,pfx_z,tot_movement,plate_x,plate_z_norm,effective_speed,release_spin_rate,estimated_ba_using_speedangle,estimated_woba_using_speedangle
2054,623474,CH,84.2,L,R,8.04,-5.52,9.752538,-1.45,1.045,83.0,1533,0.049,0.044
2354,650671,SL,81.4,L,L,11.28,-0.6,11.295946,0.14,1.61,81.2,2538,0.006,0.005
3310,650671,FF,96.2,R,L,-5.64,14.04,15.130473,-0.86,1.83,97.1,2239,0.134,0.123
8,543037,FF,97.5,R,R,10.32,18.36,21.061624,0.11,1.745,97.3,2402,0.009,0.01
51,543037,SL,88.9,R,R,-6.24,-0.24,6.244614,0.76,0.685,88.9,2523,0.057,0.051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,680694,FF,96.1,L,R,2.04,16.92,17.042535,-0.55,0.895,97.0,2161,0.197,0.177
1823,592662,SL,90.2,L,L,-1.68,9.12,9.273446,0.48,0.885,90.5,2343,0.586,1.171
2288,592662,FF,94.6,R,L,-6.36,19.08,20.112086,0.38,1.675,94.5,2275,0.337,0.453
1726,680694,FF,96.7,L,R,-0.24,15.48,15.48186,-0.38,1.415,97.7,2286,0.34,0.545


In [64]:
def get_pitcher_data(MLBID):
    pitcher_data = new_cleandf.loc[(new_cleandf['pitcher'] == MLBID)]
    pitcher_data = pitcher_data.drop(['pitcher'], axis=1)
    pitcher_data.to_csv('C:/Users/Brandon Loesch/Desktop/Machine Learning/Project/data/' + str(MLBID) + '_data_2022.csv')
    return pitcher_data

In [65]:
get_pitcher_data(434378) #Justin Verlander

,pitch_type,release_speed,stand,p_throws,pfx_x,pfx_z,tot_movement,plate_x,plate_z_norm,effective_speed,release_spin_rate,estimated_ba_using_speedangle,estimated_woba_using_speedangle
907,FF,95.6,L,R,10.08,17.4,20.108864,0.16,2.035,94.7,2444,0.211,0.427
1744,FF,95.3,R,R,8.04,20.4,21.927189,0.05,2.4,94.1,2497,0.209,0.391
3511,FF,94.9,L,R,11.04,17.28,20.505609,-0.37,1.825,93.9,2412,0.637,0.604
66,CU,79.4,L,R,-5.04,-15.12,15.937879,-0.21,0.35,78.6,2725,0.069,0.074
465,FF,95.3,R,R,11.76,18.96,22.310966,-0.28,2.5,94.4,2444,0.08,0.072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,FF,96.2,R,R,7.92,16.68,18.464799,-0.9,1.625,96.3,2400,0.003,0.003
841,SL,90.4,R,R,-5.52,6.36,8.421401,0.99,1.735,90.4,2422,0.871,0.829
1030,FF,95.5,R,R,9.36,19.44,21.575987,0.48,1.68,95.3,2359,0.037,0.061
1401,FF,95.3,R,R,14.04,18.24,23.017802,0.16,1.63,95.1,2382,0.346,0.311
